In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.test_suite import TestSuite
from checklist.perturb import Perturb


In [2]:
editor = checklist.editor.Editor()

# Sentiment analysis

In [3]:
"""
MARCO'S LOADING
"""

"""
from checklist.pred_wrapper import PredictorWrapper

import sys
sys.path.appendnd('/home/marcotcr/work/ml-tests/')
from mltests import model_wrapper
sentiment = model_wrapper.ModelWrapper()
wrapped_pp = PredictorWrapper.wrap_softmax(sentiment.predict_proba)
import csv
r = csv.DictReader(open('/home/marcotcr/datasets/airline/Tweets.csv'))
labels = []
confs = []
airlines = []
tdata = []
reasons = []
for row in r:
    sentiment, conf, airline, text = row['airline_sentiment'], row['airline_sentiment_confidence'], row['airline'], row['text']
    labels.append(sentiment)
    confs.append(conf)
    airlines.append(airline)
    tdata.append(text)
    reasons.append(row['negativereason'])

mapping = {'negative': 0, 'positive': 2, 'neutral': 1}
labels = np.array([mapping[x] for x in labels]).astype(int)


nlp = spacy.load('en_core_web_sm')

sentences = tdata
parsed_data = list(nlp.pipe(sentences))
"""

"\nfrom checklist.pred_wrapper import PredictorWrapper\n\nimport sys\nsys.path.appendnd('/home/marcotcr/work/ml-tests/')\nfrom mltests import model_wrapper\nsentiment = model_wrapper.ModelWrapper()\nwrapped_pp = PredictorWrapper.wrap_softmax(sentiment.predict_proba)\nimport csv\nr = csv.DictReader(open('/home/marcotcr/datasets/airline/Tweets.csv'))\nlabels = []\nconfs = []\nairlines = []\ntdata = []\nreasons = []\nfor row in r:\n    sentiment, conf, airline, text = row['airline_sentiment'], row['airline_sentiment_confidence'], row['airline'], row['text']\n    labels.append(sentiment)\n    confs.append(conf)\n    airlines.append(airline)\n    tdata.append(text)\n    reasons.append(row['negativereason'])\n\nmapping = {'negative': 0, 'positive': 2, 'neutral': 1}\nlabels = np.array([mapping[x] for x in labels]).astype(int)\n\n\nnlp = spacy.load('en_core_web_sm')\n\nsentences = tdata\nparsed_data = list(nlp.pipe(sentences))\n"

In [4]:
"""
TONGSHUANG'S LOADING
"""


import sys
import os
import pandas as pd
pd.set_option('display.max_colwidth', 180)
import spacy
mltest_path = os.path.abspath(os.path.expanduser("~/sourcetree/ml-tests"))
sys.path.append(mltest_path)
mlfeedback_path = os.path.abspath(os.path.expanduser("~/sourcetree/ml_feedback"))
sys.path.append(mlfeedback_path)

from ml_feedback.analysis.sentiment_utils import *
# get the model
model_names = {
    "orig": "imdb-iclr_orig"
}
from ml_feedback.analysis.sentiment_utils import *
models = load_models_by_name(model_names)

from checklist.pred_wrapper import PredictorWrapper
sentiment = models["orig"]
wrapped_pp = PredictorWrapper.wrap_softmax(sentiment.predict_proba)


from ml_feedback.analysis.sentiment_utils import *

df = read_one_dataset("semeval_iclr_nonbinary")
if len(df) > 2000:
    df = df.sample(2000)
nlp = spacy.load('en_core_web_sm')
sentences = df.sentence.tolist()
parsed_data = list(nlp.pipe(sentences))
labels = df.label.tolist()
parsed_data[0]

/home/wtshuang/datasets/models/imdb-iclr_orig
Namespace(adam_epsilon=1e-08, cache_dir='', config_name='', data_dir='/home/wtshuang/datasets/raw/glue_data//imdb/iclr_orig', device=device(type='cuda'), do_eval=True, do_lower_case=True, do_train=True, eval_all_checkpoints=False, evaluate_during_training=False, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=1, label_list='None', learning_rate=5e-05, local_rank=-1, logging_steps=50, max_grad_norm=1.0, max_seq_length=512, max_steps=-1, model_name_or_path='bert-base-uncased', model_type='bert', n_gpu=2, no_cuda=False, num_train_epochs=10.0, output_dir='//home/wtshuang/datasets/models//imdb-iclr_orig/', output_mode='classification', overwrite_cache=True, overwrite_output_dir=True, per_gpu_eval_batch_size=8, per_gpu_train_batch_size=8, save_steps=1000000000000, seed=42, server_ip='', server_port='', task_name='sst-2', tokenizer_name='', train_batch_size=16, warmup_steps=0, weight_decay=0.0)


Al Michaels said he never saw Frank Gifford lose his cool in 12 yrs working alongside him. Poised amid TV madness https://t.co/uvBolxzwW6

In [5]:
def new_pp(data):
    margin_neutral = 1/3.
    mn = margin_neutral / 2.
    pr = wrapped_pp(data)[1][:, 1]
    pp = np.zeros((pr.shape[0], 3))
    neg = pr < 0.5 - mn
    pp[neg, 0] = 1 - pr[neg]
    pp[neg, 2] = pr[neg]
    pos = pr > 0.5 + mn
    pp[pos, 0] = 1 - pr[pos]
    pp[pos, 2] = pr[pos]
    neutral_pos = (pr >= 0.5) * (pr < 0.5 + mn)
    pp[neutral_pos, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_pos] - 0.5)
    pp[neutral_pos, 2] = 1 - pp[neutral_pos, 1]
    neutral_neg = (pr < 0.5) * (pr > 0.5 - mn)
    pp[neutral_neg, 1] = 1 - (1 / margin_neutral) * np.abs(pr[neutral_neg] - 0.5)
    pp[neutral_neg, 0] = 1 - pp[neutral_neg, 1]
    preds = np.argmax(pp, axis=1)
    return preds, pp
    

In [6]:
from checklist.test_suite import TestSuite
suite = TestSuite()

# MFT

In [7]:
air_noun = ['flight', 'seat', 'pilot', 'staff', 'service', 'customer service', 'aircraft', 'plane', 'food', 'cabin crew', 'company', 'airline', 'crew']
editor.add_lexicon('air_noun', air_noun)
pos_adj = ['good', 'great', 'excellent', 'amazing', 'extraordinary', 'beautiful', 'fantastic', 'nice', 'incredible', 'exceptional', 'awesome', 'perfect', 'fun', 'happy', 'adorable', 'brilliant', 'exciting', 'sweet', 'wonderful']
neg_adj = ['awful', 'bad', 'horrible', 'weird', 'rough', 'lousy', 'unhappy', 'average', 'difficult', 'poor', 'sad', 'frustrating', 'hard', 'lame', 'nasty', 'annoying', 'boring', 'creepy', 'dreadful', 'ridiculous', 'terrible', 'ugly', 'unpleasant']
neutral_adj = ['American', 'international',  'commercial', 'British', 'private', 'Italian', 'Indian', 'Australian', 'Israeli', ]
editor.add_lexicon('pos_adj', pos_adj, overwrite=True)
editor.add_lexicon('neg_adj', neg_adj, overwrite=True )
editor.add_lexicon('neutral_adj', neutral_adj, overwrite=True)
pos_verb_present = ['like', 'enjoy', 'appreciate', 'love',  'recommend', 'admire', 'value', 'welcome']
neg_verb_present = ['hate', 'dislike', 'regret',  'abhor', 'dread', 'despise' ]
neutral_verb_present = ['see', 'find']
pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 'valued', 'welcomed']
neg_verb_past = ['hated', 'disliked', 'regretted',  'abhorred', 'dreaded', 'despised']
neutral_verb_past = ['saw', 'found']
editor.add_lexicon('pos_verb_present', pos_verb_present, overwrite=True)
editor.add_lexicon('neg_verb_present', neg_verb_present, overwrite=True)
editor.add_lexicon('neutral_verb_present', neutral_verb_present, overwrite=True)
editor.add_lexicon('pos_verb_past', pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb_past', neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb_past', neutral_verb_past, overwrite=True)
editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)
editor.add_lexicon('neg_verb', neg_verb_present + neg_verb_past, overwrite=True)
editor.add_lexicon('neutral_verb', neutral_verb_present + neutral_verb_past, overwrite=True)

## Words in context

In [17]:
t = editor.template('{it} {air_noun} {be} {pos_adj}.', it=['The', 'This', 'That'], be=['is', 'was'], labels=2, save=True)
t += editor.template('{it} {be} {a:pos_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'], labels=2, save=True)
#t += editor.template('{i} {pos_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'], labels=2, save=True)
#t += editor.template('{it} {air_noun} {be} {neg_adj}.', it=['That', 'This', 'The'], be=['is', 'was'], labels=0, save=True)
#t += editor.template('{it} {be} {a:neg_adj} {air_noun}.', it=['It', 'This', 'That'], be=['is', 'was'], labels=0, save=True)
#t += editor.template('{i} {neg_verb} {the} {air_noun}.', i=['I', 'We'], the=['this', 'that', 'the'], labels=0, save=True)
# equivalent to:
# test = MFT(t.data, labels=t.labels, templates=t.templates)
test = MFT(**t)
suite.add(test, 'Sentiment-laden words in context', 'Vocabulary', 'Use positive and negative verbs and adjectives with airline nouns such as seats, pilot, flight, etc. E.g. "This was a bad flight"')


Exception: There is already a test named Sentiment-laden words in context suite. Run with overwrite=True to overwrite

In [9]:
test.run(new_pp, n=200)
test.visual_summary()

Evaluating:   0%|          | 0/25 [00:00<?, ?it/s]

Predicting 200 examples


Evaluating: 100%|██████████| 25/25 [00:02<00:00,  9.71it/s]


TestSummarizer(stats={'npassed': 200, 'nfailed': 0, 'nfiltered': 0}, summarizer={'name': None, 'description': …

In [21]:
suite.visual_summary_by_name('Sentiment-laden words in context')

TestSummarizer(stats={'npassed': 200, 'nfailed': 0, 'nfiltered': 0}, summarizer={'name': 'Sentiment-laden word…

In [41]:
suite.visual_summary_table(capabilities=["robust"])

SuiteSummarizer(stats={'npassed': 0, 'nfailed': 0, 'nfiltered': 0})

## Intensifiers and reducers

In [27]:
intens_adj = ['very', 'really', 'absolutely', 'truly', 'extremely', 'quite', 'incredibly', 'amazingly', 'especially', 'exceptionally', 'unbelievably', 'utterly', 'exceedingly', 'rather', 'totally', 'particularly']
intens_verb = [ 'really', 'absolutely', 'truly', 'extremely',  'especially',  'utterly',  'totally', 'particularly', 'highly', 'definitely', 'certainly', 'genuinely', 'honestly', 'strongly', 'sure', 'sincerely']

In [28]:
monotonic_label = Expect.monotonic(increasing=True, tolerance=0.1)
non_neutral_pred = lambda pred, *args, **kwargs: pred != 1
monotonic_label = Expect.slice_pairwise(monotonic_label, non_neutral_pred)

In [29]:
t = editor.template(['{it} {be} {a:pos_adj} {air_noun}.', '{it} {be} {a:intens} {pos_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500, save=True)
#t += editor.template(['{i} {pos_verb} {the} {air_noun}.', '{i} {intens} {pos_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500, save=True)
#t += editor.template(['{it} {be} {a:neg_adj} {air_noun}.', '{it} {be} {a:intens} {neg_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500, save=True)
#t += editor.template(['{i} {neg_verb} {the} {air_noun}.', '{i} {intens} {neg_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500, save=True)
t.data[:5]


[['That is an extraordinary aircraft.',
  'That is an exceptionally extraordinary aircraft.'],
 ['That is an incredible crew.', 'That is a rather incredible crew.'],
 ['It was a good seat.', 'It was an especially good seat.'],
 ['It is an adorable cabin crew.', 'It is an especially adorable cabin crew.'],
 ['This is an incredible flight.',
  'This is an exceedingly incredible flight.']]

In [34]:
t = editor.template(['{it} {be} {a:pos_adj} {air_noun}.', '{it} {be} {a:intens} {pos_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500, save=True)
t += editor.template(['{i} {pos_verb} {the} {air_noun}.', '{i} {intens} {pos_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500, save=True)
t += editor.template(['{it} {be} {a:neg_adj} {air_noun}.', '{it} {be} {a:intens} {neg_adj} {air_noun}.'] , intens=intens_adj, it=['It', 'This', 'That'], be=['is', 'was'], nsamples=500, save=True)
t += editor.template(['{i} {neg_verb} {the} {air_noun}.', '{i} {intens} {neg_verb} {the} {air_noun}.'], intens=intens_verb, i=['I', 'We'], the=['this', 'that', 'the'], nsamples=500, save=True)
test = DIR(t.data, monotonic_label, templates=t.templates)
description = '''Test is composed of pairs of sentences (x1, x2), where we add an intensifier
such as "really",or "very" to x2 and expect the confidence to NOT go down (with tolerance=0.1). e.g.:
We disregard cases where the prediction of x1 is neutral.
'''
test.run(new_pp)
test.visual_summary()


Predicting 4000 examples


Evaluating: 100%|██████████| 500/500 [00:48<00:00, 10.41it/s]


TestSummarizer(stats={'npassed': 1913, 'nfailed': 80, 'nfiltered': 0}, summarizer={'name': None, 'description'…

In [36]:
description = '''Test is composed of pairs of sentences (x1, x2), where we add an intensifier
such as "really",or "very" to x2 and expect the confidence to NOT go down (with tolerance=0.1). e.g.:
We disregard cases where the prediction of x1 is neutral.
'''
suite.add(test, 'intensifiers', 'Vocabulary', description)
suite.visual_summary_by_name("intensifiers")

TestSummarizer(stats={'npassed': 1913, 'nfailed': 80, 'nfiltered': 0}, summarizer={'name': 'intensifiers', 'de…

In [22]:
suite.summary()

Vocabulary

Sentiment-laden words in context
Test cases:      2964
Fails (rate):    1 (0.0%)

Example fails:
0.1 0.9 0.0 That seat was extraordinary.
----


intensifiers
Test cases:      2000
After filtering: 1997 (99.8%)
Fails (rate):    88 (4.4%)

Example fails:
0.1 0.0 0.9 It is an average staff.
1.0 0.0 0.0 It is an incredibly average staff.

----
1.0 0.0 0.0 We dread this flight.
0.3 0.7 0.0 We particularly dread this flight.

----
1.0 0.0 0.0 It is a sad crew.
0.0 0.0 1.0 It is a quite sad crew.

----






In [15]:
monotonic_label = Expect.monotonic(increasing=True, tolerance=0.1)
non_neutral_pred = lambda pred, *args, **kwargs: pred != 1
monotonic_label = Expect.slice_pairwise(monotonic_label, non_neutral_pred)


reducer_adj = ['somewhat', 'kinda', 'mostly', 'probably', 'generally', 'reasonably', 'a little', 'a bit', 'slightly']
monotonic_label_down = Expect.monotonic(increasing=False, tolerance=0.1)
monotonic_label_down = Expect.slice_pairwise(monotonic_label_down, non_neutral_pred)

data = editor.template(['{it} {air_noun} {be} {pos_adj}.', '{it} {air_noun} {be} {red} {pos_adj}.'] , red=reducer_adj, it=['The', 'This', 'That'], be=['is', 'was'], nsamples=1000)
data += editor.template(['{it} {air_noun} {be} {neg_adj}.', '{it} {air_noun} {be} {red} {neg_adj}.'] , red=reducer_adj, it=['The', 'This', 'That'], be=['is', 'was'], nsamples=1000)
test = DIR(data, monotonic_label_down)
test.run(new_pp)
test.summary(3)
test.visual_summary()


Predicting 4000 examples


Evaluating: 100%|██████████| 500/500 [00:54<00:00,  9.25it/s]

Test cases:      2000
After filtering: 53 (2.6%)
Fails (rate):    15 (28.3%)

Example fails:
2 (0.8) The flight was aggressive.
2 (1.0) The flight was mostly aggressive.

----
2 (0.8) The flight was aggressive.
2 (1.0) The flight was somewhat aggressive.

----
2 (0.7) This plane was average.
2 (1.0) This plane was reasonably average.

----


In [3]:
pos_verb_present = ['like', 'enjoy', 'appreciate', 'love',  'recommend', 'admire', 'value', 'welcome']
pos_verb_past = ['liked', 'enjoyed', 'appreciated', 'loved', 'admired', 'valued', 'welcomed']
editor.add_lexicon('pos_verb', pos_verb_present+ pos_verb_past, overwrite=True)

#template_strs, tagged_keys, tag_dict, bert_suggests = 
editor.visual_suggest(
    '{i} {mask} {pos_verb} {a:mask}.', 
    i=['I', 'We'], 
    the=['this', 'that', 'the']
)

TemplateEditor(bert_suggests=[('always', 'interview'), ('always', 'opinion'), ('also', 'explanation'), ('alway…

In [10]:
editor.temp_selects

[('really', 'lot'), ('just', 'lot'), ('really', 'book')]

In [3]:
comp_pairs = [('better', 'worse'), ('older', 'younger'), ('smarter', 'dumber'), ('taller', 'shorter'), ('bigger', 'smaller'), ('stronger', 'weaker'), ('faster', 'slower'), ('darker', 'lighter'), ('richer', 'poorer'), ('happier', 'sadder'), ('louder', 'quieter'), ('warmer', 'colder')]
comp_pairs = list(set(comp_pairs + [(x[1], x[0]) for x in comp_pairs]))


#template_strs, tagged_keys, tag_dict, bert_suggests = 
editor.visual_suggest(
    {
        "C": '{first_name} is {a:comp[0]} {mask}.',
        "Q": 'Who is {comp[1]}?',
        "A": '{first_name}',
    },
    comp=comp_pairs,
)

TemplateEditor(bert_suggests=['version', 'picture', 'story', 'day', 'game', 'one', 'language', 'place', 'case'…

In [8]:
comp_pairs = [('better', 'worse'), ('older', 'younger'), ('smarter', 'dumber'), ('taller', 'shorter'), ('bigger', 'smaller'), ('stronger', 'weaker'), ('faster', 'slower'), ('darker', 'lighter'), ('richer', 'poorer'), ('happier', 'sadder'), ('louder', 'quieter'), ('warmer', 'colder')]
comp_pairs = list(set(comp_pairs + [(x[1], x[0]) for x in comp_pairs]))


#template_strs, tagged_keys, tag_dict, bert_suggests = 
editor.visual_suggest((
    'Is {first_name1} {last_name1} {a:mask}?',
    'Is {first_name2} {last_name2} {mask}?',
))

TemplateEditor(bert_suggests=['Muslim', 'innocent', 'gay', 'homosexual', 'dead', 'Jewish', 'suspect', 'pregnan…

In [14]:
editor.temp_selects

['innocent', 'Muslim', 'gay']

# QQP

In [1]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.mft import MFT
from checklist.inv_dir import INV, DIR
from checklist.expect import Expect
import numpy as np
import spacy
from checklist.perturb import Perturb

In [ ]:
"""
MARCO'S LOADING
"""


from checklist.pred_wrapper import PredictorWrapper

import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import model_wrapper
model = model_wrapper.ModelWrapper()
new_pp = PredictorWrapper.wrap_softmax(model.predict_proba)


qs = []
labels = []
all_questions = set()
for x in open('/home/marcotcr/datasets/glue/glue_data/QQP/dev.tsv').readlines()[1:]:
    try:
        q1, q2, label = x.strip().split('\t')[3:]
    except:
        print(x)
        continue
    all_questions.add(q1)
    all_questions.add(q2)
    qs.append((q1, q2))
    labels.append(label)
labels = np.array(labels).astype(int)

import pickle
spacy_map =  pickle.load(open('/home/marcotcr/tmp/processed_qqp.pkl', 'rb'))
parsed_qs = [(spacy_map[q[0]], spacy_map[q[1]]) for q in qs]


In [24]:
"""
TONGSHUANG'S LOADING
"""

import sys
import os
import pandas as pd
pd.set_option('display.max_colwidth', 180)

mltest_path = os.path.abspath(os.path.expanduser("~/sourcetree/ml-tests"))
sys.path.append(mltest_path)
mlfeedback_path = os.path.abspath(os.path.expanduser("~/sourcetree/ml_feedback"))
sys.path.append(mlfeedback_path)

from ml_feedback.analysis.qqp_utils import *


# get the model
model_names = {
    "qqp": "QQP-all",
}
from ml_feedback.analysis.qqp_utils import *
models = load_models_by_name(model_names)

from checklist.pred_wrapper import PredictorWrapper
model = models["qqp"]
new_pp = PredictorWrapper.wrap_softmax(model.predict_proba)



# getting the data

df = read_one_dataset("quora", max_size=2000)
parsed_qs = list(zip(
    nlp.pipe(df.question1.tolist()), 
    nlp.pipe(df.question2.tolist())
))
labels = df.is_duplicate.tolist()
parsed_qs[0]

/home/wtshuang/datasets/models/QQP-all
Namespace(adam_epsilon=1e-08, cache_dir='', config_name='', data_dir='/home/wtshuang/datasets/raw/glue_data//QQP/all', device=device(type='cuda'), do_eval=True, do_lower_case=True, do_train=True, eval_all_checkpoints=False, evaluate_during_training=False, fp16=False, fp16_opt_level='O1', gradient_accumulation_steps=1, label_list='None', learning_rate=2e-05, local_rank=-1, logging_steps=50, max_grad_norm=1.0, max_seq_length=512, max_steps=-1, model_name_or_path='bert-base-uncased', model_type='bert', n_gpu=2, no_cuda=False, num_train_epochs=3.0, output_dir='//home/wtshuang/datasets/models//QQP-all/', output_mode='classification', overwrite_cache=True, overwrite_output_dir=True, per_gpu_eval_batch_size=8, per_gpu_train_batch_size=8, save_steps=1000000000000, seed=42, server_ip='', server_port='', task_name='qqp', tokenizer_name='', train_batch_size=16, warmup_steps=0, weight_decay=0.0)


In [25]:
editor = checklist.editor.Editor()
nlp = spacy.load('en_core_web_sm')

# Vocabulary

In [5]:
professions = editor.suggest('{first_name} works as {a:bert}.')[:30]
print(', '.join(professions))

accountant, engineer, journalist, attorney, editor, artist, architect, actor, interpreter, analyst, intern, assistant, escort, organizer, economist, actress, administrator, investigator, agent, investor, secretary, author, entrepreneur, executive, advisor, auditor, educator, instructor, employee, adviser


In [20]:
#professions = editor.suggest('{first_name} works as {a:bert}.')[:30]
#professions += editor.suggest('{first_name} {last_name} works as {a:bert}.')[:30]
#professions = list(set(professions))
professions = []
# professions
other_nouns = ['player', 'person', 'friend', 'kid', 'candidate']
nouns = list(set(professions + other_nouns))

In [22]:
#print(', '.join(editor.suggest('Is {first_name} {last_name} {a:bert} {noun}?', noun=nouns)[:50]))
adjs = ['effective', 'actual', 'American', 'active', 'honest', 'excellent', 'elite', 'acomplished', 'official', 'outstanding', 'experienced', 'independent', 'international', 'aspiring', 'average', 'good', 'amazing', 'exceptional', 'successful', 'accredited', 'English', 'real', 'bad', 'terrible', 'fake', 'unusual', 'influential', 'incompetent']


In [23]:
data = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Is {first_name} {last_name} {a:adj} {noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(data, labels=0)
test.run(new_pp)
test.summary(n=3)

Predicting 999 examples


Evaluating: 100%|██████████| 125/125 [00:13<00:00,  9.33it/s]

Test cases:      999
Fails (rate):    415 (41.5%)

Example fails:
0.7 ('Is Ryan Long a kid?', 'Is Ryan Long an effective kid?')
----
1.0 ('Is Danielle Price a friend?', 'Is Danielle Price an average friend?')
----
1.0 ('Is Amy King a player?', 'Is Amy King an average player?')
----


In [24]:
test.visual_summary()

TestSummarizer(stats={'nTested': 999, 'nFailed': 415, 'nFiltered': 0}, summarizer={'name': 'TESTNAME_PLACEHOLD…

## Change name, loc, number in only one where orig prediction is duplicate

In [28]:
def wrap_apply_to_each(fn, both=False, *args, **kwargs):
    def new_fn(qs, *args, **kwargs):
        q1, q2 = qs
        ret = []
        fnq1 = fn(q1, *args, **kwargs)
        fnq2 = fn(q2, *args, **kwargs)
        if type(fnq1) != list:
            fnq1 = [fnq1]
        if type(fnq2) != list:
            fnq2 = [fnq2]
        ret.extend([(x, str(q2)) for x in fnq1])
        ret.extend([(str(q1), x) for x in fnq2])
        if both:
            ret.extend([(x, x2) for x, x2 in itertools.product(fnq1, fnq2)])
        return [x for x in ret if x[0] and x[1]]
    return new_fn
data = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_names), nsamples=1500, first_only=True)
expect_fn = Expect.eq(0)
expect_fn = Expect.slice_orig(expect_fn, lambda orig, *args: orig == 1)
test = DIR(data, expect_fn)
test.run(new_pp)
test.summary(3)


Predicting 1332 examples


Evaluating: 100%|██████████| 167/167 [00:16<00:00, 10.42it/s]

Test cases:      82
After filtering: 43 (52.4%)
Fails (rate):    41 (95.3%)

Example fails:
1.0 ('What will happen if Donald Trump became the president of America ?', 'What will happen if Donald Trump wins the election ?')
1.0 ('What will happen if Donald Trump became the president of America ?', 'What will happen if Henry Trump wins the election ?')
1.0 ('What will happen if Donald Trump became the president of America ?', 'What will happen if Jose Trump wins the election ?')

----
1.0 ('According to the Bible , is the second coming of Jesus in the near future ?', 'Is the second coming near ?')
1.0 ('According to the Bible , is the second coming of Austin in the near future ?', 'Is the second coming near ?')
0.9 ('According to the Bible , is the second coming of Joseph in the near future ?', 'Is the second coming near ?')

----
1.0 ("Who runs Trump 's Twitter account ? Does Donald Trump write his own tweets ?", 'Does Donald Trump really write his own tweets ?')
1.0 ("Who runs Trump 's

In [29]:
test.visual_summary()

TestSummarizer(stats={'npassed': 2, 'nfailed': 41, 'nfiltered': 0}, summarizer={'name': 'TESTNAME_PLACEHOLDER'…